# Introduction
Ce projet se concentre sur la création d'un modèle de question-réponse spécialisé dans le domaine biomédical, basé sur l'architecture BERT (Bidirectional Encoder Representations from Transformers). En utilisant le jeu de données bioASK, le modèle est conçu pour comprendre des questions en langage naturel et trouver des réponses spécifiques dans un texte fourni. La démarche suit plusieurs étapes essentielles, depuis la préparation des données jusqu’à l'entraînement et la sauvegarde du modèle, afin de garantir sa précision dans le domaine médical. Ce projet démontre également l'intégration de bibliothèques modernes comme Hugging Face Transformers et TensorFlow pour le développement d'un modèle performant de question-réponse, ouvrant des perspectives dans la création d'outils d’aide à la recherche d'information spécialisée.

# Implementation du Modèle

In [ ]:
import json
from transformers import BertTokenizer
from transformers import BertTokenizerFast, TFBertForQuestionAnswering
import tensorflow as tf
import tensorflow as tf
from transformers import TFBertForQuestionAnswering


La première étape consiste à importer les modules nécessaires pour le projet, en particulier Hugging Face Transformers pour le modèle BERT et le tokenizer, ainsi que TensorFlow pour la création et l'entraînement du modèle.

In [ ]:
# Charger le fichier JSON de BIOAsk
with open("bio.json") as f:
    squad_data = json.load(f)

On initialise le tokenizer BERT, qui découpe les phrases en tokens que le modèle peut interpréter. Ce tokenizer est utilisé pour transformer les questions et les contextes en vecteurs de tokens exploitables par BERT.

In [ ]:
# Tokenizer BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [5]:

# Préparation des données (extrait question et contexte)
questions = []
contexts = []
answers = []

for article in squad_data["data"]:
    for paragraph in article["paragraphs"]:
        context = paragraph["context"]
        for qa in paragraph["qas"]:
            question = qa["question"]
            answer = qa["answers"][0]["text"]
            questions.append(question)
            contexts.append(context)
            answers.append(answer)

# Affichage du nombre d'entrées
print(f"Nombre de questions : {len(questions)}")



Nombre de questions : 4772


# Préparation des Données pour le Modèle
Cette section tokenize les questions et contextes et détermine les positions de début et de fin des réponses. Cela permet de spécifier l’emplacement exact des réponses dans le contexte, essentiel pour l’apprentissage supervisé du modèle.


In [ ]:

# Préparation des données
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
questions = []
contexts = []
start_positions = []
end_positions = []

for article in squad_data["data"]:
    for paragraph in article["paragraphs"]:
        context = paragraph["context"]
        for qa in paragraph["qas"]:
            question = qa["question"]
            answer = qa["answers"][0]
            answer_text = answer["text"]
            start_idx = answer["answer_start"]
            end_idx = start_idx + len(answer_text)

            # Tokenisation avec le tokenizer rapide
            inputs = tokenizer(question, context, return_tensors="tf", truncation=True, padding="max_length", max_length=384)

            # Obtenir les positions de début et de fin
            start_pos = inputs.char_to_token(0, start_idx)
            end_pos = inputs.char_to_token(0, end_idx - 1)
            
            # Vérifier si les positions ne sont pas None
            if start_pos is not None and end_pos is not None:
                start_positions.append(start_pos)
                end_positions.append(end_pos)
                questions.append(question)
                contexts.append(context)


## Conversion des Données en Tenseurs
Une fois les positions de début et de fin établies, elles sont converties en tenseurs TensorFlow, afin d’être prêtes pour l’entraînement du modèle.

In [6]:

# Conversion en tenseurs pour l'entraînement
start_positions = tf.convert_to_tensor(start_positions)
end_positions = tf.convert_to_tensor(end_positions)
inputs = tokenizer(questions, contexts, return_tensors="tf", padding=True, truncation=True)


# Chargement , Compilation et Entraînement du Modèle



Ici, le modèle TFBertForQuestionAnswering est configuré pour une tâche de question-réponse. Il est compilé avec l’optimiseur Adam et une fonction de perte pour les positions de début et de fin, assurant une optimisation correcte pendant l’entraînement.

In [ ]:

# Charger le modèle
model = TFBertForQuestionAnswering.from_pretrained("bert-base-uncased")

In [7]:


# Compiler le modèle
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
              loss={'start_positions': 'sparse_categorical_crossentropy', 'end_positions': 'sparse_categorical_crossentropy'})

# Définir un callback pour sauvegarder le modèle à chaque epoch
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="model_epoch_{epoch}.h5",  # Nom du fichier avec numéro d'epoch
    save_weights_only=True,              # Sauvegarder uniquement les poids
    save_freq='epoch'                    # Sauvegarder à chaque epoch
)

# Entraînement avec le callback
model.fit(inputs, {"start_positions": start_positions, "end_positions": end_positions},
          epochs=4, batch_size=8, callbacks=[checkpoint_callback])


All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

Some weights or buffers of the TF 2.0 model TFBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/4


34/34 [==============================] - 600s 17s/step - loss: 6.7903 - start_logits_loss: 6.7903
Epoch 2/4
34/34 [==============================] - 644s 19s/step - loss: 6.2383 - start_logits_loss: 6.2383
Epoch 3/4
34/34 [==============================] - 618s 18s/step - loss: 6.2383 - start_logits_loss: 6.2383
Epoch 4/4
34/34 [==============================] - 481s 14s/step - loss: 6.2383 - start_logits_loss: 6.2383


# code de l'application streamlit 

In [ ]:

# Charger le modèle et le tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
model = TFBertForQuestionAnswering.from_pretrained("bert-base-uncased")

# Charger le fichier JSON de données
with open("bio.json") as f:
    squad_data = json.load(f)

# Fonction pour obtenir la réponse prédite par le modèle
def get_predicted_answer(question, context):
    # Tokenizer en gardant une longueur maximale plus flexible
    inputs = tokenizer(question, context, return_tensors="tf", truncation=True, padding="max_length", max_length=512)
    
    # Prédiction des logits de début et de fin
    outputs = model(inputs)
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    # Trouver les positions des tokens de début et de fin avec les scores les plus élevés
    start_idx = tf.argmax(start_logits, axis=1).numpy()[0]
    end_idx = tf.argmax(end_logits, axis=1).numpy()[0] + 1

    # Vérification si la position de fin est après celle du début
    if end_idx < start_idx:
        end_idx = start_idx + 1

    # Convertir les tokens en texte
    answer_tokens = inputs["input_ids"][0][start_idx:end_idx]
    answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

    # Nettoyage pour éviter les erreurs dans la réponse
    if len(answer.strip()) == 0:
        answer = "Je ne peux pas trouver une réponse appropriée dans le contexte fourni."
    
    return answer

# Interface Streamlit
st.set_page_config(page_title="Medical Question-Answering Bot", page_icon="🤖")
st.title("Medical Question-Answering Bot")

# Message dans la barre latérale
with st.sidebar:
    st.title("Paramètres")
    st.write("Entrez le contexte et la question pour obtenir une réponse basée sur le modèle de QA.")

# Prendre les entrées de l'utilisateur
user_context = st.text_area("Entrez le texte ou le contexte ici...")
user_question = st.text_input("Tapez votre question ici...")

# Si le contexte et la question sont fournis, obtenir la réponse
if user_context and user_question:
    predicted_answer = get_predicted_answer(user_question, user_context)

    # Afficher la réponse prédite
    st.write(f"Réponse : {predicted_answer}")


# Conclusion
Ce projet propose une méthode complète pour développer un système de question-réponse biomédical, basé sur BERT, adapté aux besoins de recherche de réponses précises dans des corpus spécialisés. Grâce à un processus structuré, allant de la préparation des données à l'entraînement du modèle, ce système démontre l'efficacité de l'apprentissage profond dans les applications de traitement du langage naturel. Pour l'avenir, une exploration d'autres architectures plus avancées ou des optimisations spécifiques au domaine médical pourraient encore améliorer les performances, rendant cet outil d'autant plus utile pour les chercheurs, les médecins et les étudiants en sciences de la santé.

#Références 


[1] BERT for Question Answering on BioASQ (2020)
Auteurs: Fu, Djoko, Mansor, and Slater


[2] BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding (2018) 
Auteurs: Jacob Devlin, Ming-Wei Chang, Kenton Lee, Kristina Toutanova



[3]BERT for Extractive Question Answering on SQuAD(2018)
Auteurs: Jacob Devlin, Ming-Wei Chang, Kenton Lee, Kristina Toutanova


[4]Transformers for Biomedical Question Answering: A Systematic Review (2021)
Auteurs:Y. Rasmy, J. Li, and M. Liu


[5]Latent Structure in Transformer-based QA Systems (2020)
Auteurs:des chercheurs spécialisés dans les modèles de transformers

[6]GitHub - BERT BioASQ Q/A Mahdiar-Khodabakhshi

[7]Kaggle - BioASQ Dataset

